In [1]:
!nvidia-smi

Sun Feb 15 11:29:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install numba

In [3]:
import numpy as np
from numba import cuda
import math
import time
import os

# Create folders
os.makedirs("input_csv", exist_ok=True)
os.makedirs("output_csv", exist_ok=True)

# Generate 60 CSV files with random numbers
for i in range(60):
    data = np.random.randint(1, 100, 2000)
    np.savetxt(f"input_csv/file_{i}.csv", data, fmt="%d")

# CUDA Kernel
@cuda.jit
def square_kernel(input_arr, output_arr):
    idx = cuda.grid(1)
    if idx < input_arr.size:
        output_arr[idx] = input_arr[idx] * input_arr[idx]

start = time.time()

# Process all CSV files
for file in os.listdir("input_csv"):
    data = np.loadtxt(f"input_csv/{file}")

    d_input = cuda.to_device(data)
    d_output = cuda.device_array_like(data)

    threads_per_block = 256
    blocks = math.ceil(data.size / threads_per_block)

    square_kernel[blocks, threads_per_block](d_input, d_output)

    result = d_output.copy_to_host()
    np.savetxt(f"output_csv/{file}", result, fmt="%d")

end = time.time()

print("Processed 60 CSV files")
print("Time Taken:", end - start, "seconds")


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:680: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Processed 60 CSV files
Time Taken: 1.5143680572509766 seconds


In [4]:
!nvidia-smi

Sun Feb 15 11:37:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             33W /  250W |     263MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----